Для начала мы хотим опреледить значимые для нашего корпуса слова. Для этого мы сравним частотности токенов нашего корпуса и какого-нибудь нейтрального. В качестве нейтрального корпуса мы задействуем кусок ГИКРЯ, по размеру равный нашему корпусу.

Мы знаем, что в нашем корпусе 3953288 токенов и имеем частотный список.

In [132]:
import os
import pickle
from collections import Counter


with open(os.path.join("utils", "food_corpus_word_counter.pickle"), "rb") as fd:
    food_corpus_counter = pickle.load(fd)

In [2]:
import sys
import ufal.udpipe

ud_model = ufal.udpipe.Model.load("russian-syntagrus-ud-2.4-190531.udpipe")
pipe = ufal.udpipe.Pipeline(ud_model, "tokenize", "tag", "parse", "conllu")

In [86]:
with open("ruwac-parsed.out", "r") as file:
    corpus = file.read(120500000)

In [87]:
print(len(corpus.split("\n")))

3956622


In [92]:
from tqdm import tqdm

neutral_counter = Counter()

tokens = corpus.split("\n")
for token in tqdm(tokens):
    if len(token.split("\t")) > 4:
        tag = pipe.process(token.split("\t")[3]).split("\t")[3]
        neutral_counter.update({token.split("\t")[3] + "_" + tag: 1})

100%|██████████| 3956622/3956622 [20:19<00:00, 3244.38it/s]


In [93]:
with open(os.path.join("utils", "neutral_corpus_word_counter.pickle"), "wb") as fd:
    food_corpus_counter = pickle.dump(neutral_counter, fd)

In [95]:
neutral_counter.most_common(10)

[(',_PUNCT', 311870),
 ('._PUNCT', 204368),
 ('и_CCONJ', 103893),
 ('в_ADP', 92811),
 ('не_PART', 60135),
 ('я_PRON', 49189),
 ('на_ADP', 47512),
 ('что_SCONJ', 43791),
 (')_PUNCT', 38694),
 ('быть_AUX', 37759)]

In [133]:
for key in list(food_corpus_counter):
    if key in neutral_counter and ("_NOUN" in key or "_VERB" in key):
        food_corpus_counter[key] = food_corpus_counter[key] / neutral_counter[key]
    else:
        food_corpus_counter.pop(key)

Нас интересуют только те токены, которые существенно существеннее в нашем корпусе, чем в нейтральном. Выберем некоторое количество наиболее особенных

In [217]:
food_corpus_counter.most_common(10)

[('телятина_NOUN', 564.0),
 ('вспомянуть_VERB', 441.0),
 ('сервировка_NOUN', 309.0),
 ('гребешка_NOUN', 289.0),
 ('пр._NOUN', 286.0),
 ('котлет_NOUN', 262.0),
 ('грудок_NOUN', 251.0),
 ('хостес_NOUN', 229.0),
 ('морс_NOUN', 222.66666666666666),
 ('шеф-повар_NOUN', 217.5)]

In [205]:
keys = food_corpus_counter.most_common(700)

Найдем центроиды имеющихся ключей по еде и посервису. Получаем два центроида. Далее мы будем использовать их как отправную точку для кластеризации значимых токенов в keys.

In [206]:
from gensim.models import KeyedVectors
import numpy as np
import pymorphy2
import os

morph = pymorphy2.MorphAnalyzer()
model = KeyedVectors.load_word2vec_format(os.path.join("wv", "model.bin"), binary=True)

tags = {"NOUN": "NOUN",
       "INFN": "VERB",
       "ADJF": "ADJ"}


def get_key_centroids(key_path):
    vectors = []
    with open(key_path, "r") as file:
        keys = file.read().split()
        print(keys)
        for i, key in enumerate(keys):
            tag = morph.parse(key)[0].tag.POS
            model_token = keys[i] + "_" + tags[tag]
            if model_token in model:
                vectors.append(model[model_token])
    return np.mean(vectors)

food_centroid = get_key_centroids(os.path.join("seeds", "food_key_extended.txt"))
service_centroid = get_key_centroids(os.path.join("seeds", "service_key_extended.txt"))

['бутерброд', 'овощ', 'рагу', 'кушанье', 'батон', 'жаркое', 'напиток', 'суп', 'алкоголь', 'десерт', 'кляр', 'повар', 'сыр', 'кетчуп', 'тарелка', 'пицца', 'готовить', 'яство', 'паста', 'состав', 'спирт', 'говядина', 'варить', 'щука', 'кофе', 'гребешок', 'борщ', 'просфора', 'ингредиент', 'зажаривать', 'гуляш', 'тефтели', 'плюшка', 'ветчина', 'пшеница', 'запеканка', 'булочка', 'хлеб', 'баранина', 'морс', 'колбаса', 'выпечка', 'лимонад', 'квас', 'соус', 'молоко', 'печень', 'сок', 'свинина', 'ликёр', 'вино', 'цезарь', 'лосось', 'тунец', 'компот', 'подливка', 'бульон', 'нектар', 'блюдо', 'пунш', 'снедь', 'сосиска', 'обед', 'шампанское', 'краб', 'питие', 'пирог', 'стряпать', 'окунь', 'ужин', 'коктейль', 'песто', 'член', 'кальмар', 'поджаривать', 'жратва', 'пища', 'рыба', 'жарить', 'цена', 'мясо', 'японский', 'пирожок', 'плов', 'яичница', 'порция', 'коньяк', 'сухарь', 'форель', 'салат', 'макароны', 'котлета', 'йогурт', 'вермут', 'винегрет', 'кухня', 'завтрак', 'кока-кола', 'лисичка', 'гренок',

In [207]:
vectorized_keys = []
vectorized_labels = []

for key in keys:
    if key[0] in model:
        vectorized_keys.append(np.array(model[key[0]]))
        vectorized_labels.append(key[0])

vectorized_keys = np.array(vectorized_keys)

In [208]:
vectorized_keys.shape

(564, 300)

Нормализуем векторизованные эмбеддинги и центроиды.

In [209]:
from sklearn.preprocessing import normalize

normalized_embs = normalize(food_centroid + service_centroid + vectorized_keys, axis=1)

In [210]:
keys_to_sort = normalized_embs[2:]
food_centroid = normalized_embs[0]
service_centroid = normalized_embs[1]

In [211]:
from sklearn.cluster import KMeans

kmeans = KMeans(n_clusters=2, init=np.array([food_centroid, service_centroid]), random_state=0).fit(keys_to_sort)
sorted_keys = kmeans.labels_
sorted_keys

/usr/local/lib/python3.6/dist-packages/sklearn/cluster/k_means_.py:969: RuntimeWarning: Explicit initial center position passed: performing only one init in k-means instead of n_init=10
  return_n_iter=True)


array([1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1,
       0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 1,
       1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 1, 1,
       0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 1, 0, 1, 1, 1, 0,
       0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 1,
       0, 1, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 1, 0, 1, 1, 1,
       0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0,
       1, 1, 1, 0, 1, 0, 0, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1, 0,
       1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0,
       1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0,
       1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0,
       0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 1, 0, 1,
       0, 1, 0, 1, 1, 0, 0, 0, 1, 1, 1, 0, 1, 1, 1, 0, 0, 1, 0, 0, 1, 1,
       0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0,

In [212]:
new_food_keys = []
new_service_keys = []

for i, class_ in enumerate(sorted_keys):
    new_food_keys.append(vectorized_labels[i]) if class_ == 0 else new_service_keys.append(vectorized_labels[i]) 

In [213]:
print("food:", new_food_keys)
print("service:", new_service_keys)

food: ['вспомянуть_VERB', 'сервировка_NOUN', 'гребешка_NOUN', 'котлет_NOUN', 'шеф-повар_NOUN', 'цыпленка_NOUN', 'угрь_NOUN', 'медальон_NOUN', 'баранина_NOUN', 'десерт_NOUN', 'мангал_NOUN', 'горшочка_NOUN', 'резервировать_VERB', 'эклер_NOUN', 'шпинат_NOUN', 'объедение_NOUN', 'стейк_NOUN', 'диванчик_NOUN', 'харчо_NOUN', 'плошка_NOUN', 'курочка_NOUN', 'кальяна_NOUN', 'заморозка_NOUN', 'орешка_NOUN', 'вареник_NOUN', 'говядина_NOUN', 'рагу_NOUN', 'ресторатор_NOUN', 'пирога_NOUN', 'мант_NOUN', 'бармен_NOUN', 'бекон_NOUN', 'банкет_NOUN', 'кляр_NOUN', 'гребешок_NOUN', 'ур_NOUN', 'закуска_NOUN', 'кальмар_NOUN', 'меню_NOUN', 'подложка_NOUN', 'кебаб_NOUN', 'маринад_NOUN', 'перечница_NOUN', 'накурить_VERB', 'сиг_NOUN', 'интерьер_NOUN', 'томат_NOUN', 'колбаска_NOUN', 'рис_NOUN', 'каперс_NOUN', 'заполненность_NOUN', 'караок_NOUN', 'соус_NOUN', 'капуччино_NOUN', 'пепельница_NOUN', 'нарекание_NOUN', 'свинина_NOUN', 'обсчитать_VERB', 'сидр_NOUN', 'сухарик_NOUN', 'суша_NOUN', 'лобио_NOUN', 'дыхать_VERB'

Ключи получились не очень, особенно у сервиса. Попробуем взять только самые близкие к центроидам токены из всех, что рассортировала модель.

In [214]:
from scipy.spatial.distance import cosine

food_cosine = Counter()
service_cosine = Counter()

for i, class_ in enumerate(sorted_keys):
    if class_ == 0:
        food_cosine.update({vectorized_labels[i]: cosine(food_centroid, keys_to_sort[i])})
    else:
        service_cosine.update({vectorized_labels[i]: cosine(service_centroid, keys_to_sort[i])})

In [221]:
food_cosine.most_common(10)

[('сметанка_NOUN', 0.8887681216001511),
 ('пропитка_NOUN', 0.8865084424614906),
 ('вспомянуть_VERB', 0.8723135590553284),
 ('кура_NOUN', 0.8707289546728134),
 ('четвертинка_NOUN', 0.8680258989334106),
 ('нарезка_NOUN', 0.8620701432228088),
 ('котлетка_NOUN', 0.8579867482185364),
 ('вытяжка_NOUN', 0.8513587862253189),
 ('лучок_NOUN', 0.8502956330776215),
 ('старательность_NOUN', 0.8368657678365707)]

In [220]:
service_cosine.most_common(10)

[('подливать_VERB', 1.2080330699682236),
 ('фаршировать_VERB', 1.1630410701036453),
 ('пенка_NOUN', 1.1565651595592499),
 ('официантка_NOUN', 1.1546566933393478),
 ('соление_NOUN', 1.1514314860105515),
 ('мидие_NOUN', 1.14872445166111),
 ('абажур_NOUN', 1.1450221538543701),
 ('оладья_NOUN', 1.1426728963851929),
 ('сельдь_NOUN', 1.1379251182079315),
 ('подавать_VERB', 1.1338455975055695)]

Все равно что-то не очень хорошо.

In [224]:
with open(os.path.join("seeds", "super_extended_food_keys.txt"), "w") as fd:
    for el in food_cosine.most_common():
        fd.write(el[0] + "\t" + str(el[1]) + "\n")
        
with open(os.path.join("seeds", "super_extended_service_keys.txt"), "w") as fd:
    for el in service_cosine.most_common():
        fd.write(el[0] + "\t" + str(el[1]) + "\n")